In [1]:
from lib.prepro import Preprocessing, SparkDataFrame, PolarsDataFrame
from tqdm.notebook import tqdm
import os
from pprint import pprint
import pandas as pd

import pathlib

In [2]:
# PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
# PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

# 컨테이너 배포 시 사용
# PATH_CCTV_DATA = './data/cctv_data/*'
# PATH_SAND_DATA = './data/sand_data/*'
# PATH_SAND_LABEL = './data/sand_label_data/*'


# 열화상 cctv 데이터
PATH_CCTV_DATA = '/home/jovyan/work/local_src/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
PATH_CCTV_LABEL_DATA = '/home/jovyan/work/local_src/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/라벨링데이터/*'

# 모래 품질 데이터
PATH_SAND_DATA = '/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
PATH_SAND_LABEL_DATA = '/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

In [3]:
prepro = Preprocessing()
sdf = SparkDataFrame()
pdf = PolarsDataFrame()

In [4]:
# # extract file path 
cctv_paths = prepro.get_all_file_paths(PATH_CCTV_DATA)
cctv_label_paths = prepro.get_all_file_paths(PATH_CCTV_LABEL_DATA)

sand_paths = prepro.get_all_file_paths(PATH_SAND_DATA)
sand_label_paths = prepro.get_all_file_paths(PATH_SAND_LABEL_DATA)

In [5]:
print(len(cctv_paths))
print(len(cctv_label_paths))
print(len(sand_paths))
print(len(sand_label_paths))

1457336
343830
490755
45353


In [6]:
# %%time
# # make dataframe
# cctv_df = pdf.get_polars_dataframe(cctv_paths)
# cctv_label_df = pdf.get_polars_dataframe(cctv_label_paths)

# sand_df = pdf.get_polars_dataframe(sand_paths)
# sand_label_df = pdf.get_polars_dataframe(sand_label_paths)

In [8]:
%%time
# make dataframe
sp_cctv_df = sdf.get_spark_dataframe(cctv_paths)
sp_cctv_label_df = sdf.get_spark_dataframe(cctv_label_paths)

sp_sand_df = sdf.get_spark_dataframe(sand_paths)
sp_sand_label_df = sdf.get_spark_dataframe(sand_label_paths)

CPU times: user 19.8 s, sys: 307 ms, total: 20.1 s
Wall time: 21.9 s


In [6]:
cctv_parquet_path = './preprocessed_data/sp_cctv_data'
cctv_label_parquet_path = './preprocessed_data/sp_cctv_label_data'

snad_parquet_path = './preprocessed_data/sp_sand_data'
sand_label_parquet_path = './preprocessed_data/sp_sand_label_data'

# cctv_parquet_path = './preprocessed_data/cctv_data/cctv.parquet'
# cctv_label_parquet_path = './preprocessed_data/cctv_label_data/cctv_label.parquet'

# snad_parquet_path = './preprocessed_data/sand_data/sand.parquet'
# sand_label_parquet_path = './preprocessed_data/sand_label_data/sand_label.parquet'

if not os.path.exists(snad_parquet_path):
    os.makedirs(cctv_parquet_path)
    os.makedirs(cctv_label_parquet_path)
    os.makedirs(snad_parquet_path)
    os.makedirs(sand_label_parquet_path)

In [23]:
%%time
cctv_df.write_parquet(cctv_parquet_path, use_pyarrow=True)

CPU times: user 559 ms, sys: 1.02 s, total: 1.58 s
Wall time: 1.85 s


In [36]:
%%time
cctv_label_df.write_parquet(cctv_label_parquet_path, use_pyarrow=True)

CPU times: user 144 ms, sys: 15.6 ms, total: 159 ms
Wall time: 220 ms


In [30]:
%%time
sand_df.write_parquet(snad_parquet_path, use_pyarrow=True)

CPU times: user 223 ms, sys: 242 ms, total: 466 ms
Wall time: 652 ms


In [37]:
%%time
sand_label_df.write_parquet(sand_label_parquet_path, use_pyarrow=True)

CPU times: user 28.7 ms, sys: 0 ns, total: 28.7 ms
Wall time: 76 ms


In [3]:
import polars as pl

In [6]:
pl_cctv_df = pl.read_parquet(cctv_parquet_path)
pl_cctv_label_df = pl.read_parquet(cctv_label_parquet_path)
pl_sand_df = pl.read_parquet(snad_parquet_path)
pl_sand_label_df = pl.read_parquet(sand_label_parquet_path)

In [9]:
repartitioned_cctv_df = sp_cctv_df.repartition(10)
coalesce_cctv_df = sp_cctv_df.coalesce(10)

In [10]:
def save_parquet(data, path, comp_type="snappy"):
    data.write \
        .mode("overwrite") \
        .option("compression", comp_type) \
        .parquet(path)


In [ ]:
save_parquet(repartitioned_cctv_df, cctv_parquet_path)

In [9]:
sp_cctv_df.limit(2).show()

+--------------------+--------------------+--------------------+------------+---------+
|           full_path|             file_id|           file_name| folder_name|file_size|
+--------------------+--------------------+--------------------+------------+---------+
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   741101|
+--------------------+--------------------+--------------------+------------+---------+



In [6]:
# make dataframe
# cctv_df = sdf.get_spark_dataframe(cctv_paths)
# cctv_label_df = sdf.get_spark_dataframe(cctv_label_paths)

# sand_df = sdf.get_spark_dataframe(sand_paths)
# sand_label_df = sdf.get_spark_dataframe(sand_label_paths)

In [7]:
sand_df.limit(2).show()

+--------------------+-------+-----------+-----------------------+---------+
|           full_path|file_id|  file_name|            folder_name|file_size|
+--------------------+-------+-----------+-----------------------+---------+
|/home/jovyan/work...|0010002|0010002.png|TS_1.모래입자크기분류_1|  3156156|
|/home/jovyan/work...|0010003|0010003.png|TS_1.모래입자크기분류_1|  3029314|
+--------------------+-------+-----------+-----------------------+---------+



### convert dataframe to parquet

In [8]:
# parquet_path = './preprocessed_data/'
# # parquet_name = 'sanddata'
# full_parquet_path = os.path.join(parquet_path, parquet_name)
# if os.path.exists(parquet_path):
#     print(full_parquet_path)
# else:
#     print('need to setting path')
# sand_parquet_path = './preprocessed_data/sand_data'
# if os.path.exists(sand_parquet_path) == False:
#     os.makedirs(sand_parquet_path)

In [9]:
# sand_df.write \
#   .mode("overwrite") \
#   .option("compression", "snappy") \
#   .parquet(sand_parquet_path)

In [10]:
# test_parquet_path = './preprocessed_data/test'
# if os.path.exists(test_parquet_path) == False:
#     os.makedirs(test_parquet_path)
# df.write.mode("overwrite").parquet('./preprocessed_data/test/test.parquet')

In [8]:
cctv_parquet_path = './preprocessed_data/cctv_data'
cctv_label_parquet_path = './preprocessed_data/cctv_label_data'

snad_parquet_path = './preprocessed_data/sand_data'
sand_label_parquet_path = './preprocessed_data/sand_label_data'

if os.path.exists(snad_parquet_path) == False:
    os.makedirs(cctv_parquet_path)
    os.makedirs(cctv_label_parquet_path)
    os.makedirs(snad_parquet_path)
    os.makedirs(sand_label_parquet_path)

In [ ]:
pd_cctv_df.to_parquet(cctv_parquet_path, compression='snappy')

In [12]:
%%time
cctv_df.repartition(100) \
    .write \
    .option("maxRecordsPerFile", 10000) \
    .option("compression", "snappy") \
    .mode("overwrite") \
    .parquet(cctv_parquet_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 34654)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

Py4JError: An error occurred while calling o239.parquet

In [ ]:
from pyspark.sql.functions import col

def save_parquet_in_chunks(data, path, chunk_size=100000):
    total_rows = data.count()
    num_chunks = (total_rows // chunk_size) + 1
    
    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, total_rows)
        chunk = data.limit(end).subtract(data.limit(start))
        
        chunk.write \
            .option("maxRecordsPerFile", 10000) \
            .option("compression", "snappy") \
            .mode("append") \
            .parquet(path)
        
save_parquet_in_chunks(cctv_df, cctv_parquet_path)


In [ ]:
cctv_df.repartition(1).write.option("maxRecordsPerFile", 1000).option("compression", comp_type).mode("overwrite").parquet(cctv_parquet_path)

In [9]:
cctv_df.limit(2).show()

+--------------------+--------------------+--------------------+------------+---------+
|           full_path|             file_id|           file_name| folder_name|file_size|
+--------------------+--------------------+--------------------+------------+---------+
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   741101|
+--------------------+--------------------+--------------------+------------+---------+



In [8]:
def save_parquet(data, path, comp_type="snappy"):
    data.write \
        .mode("overwrite") \
        .option("compression", comp_type) \
        .parquet(path)


def save_parquet_in_chunks(data, db_path, chunk_size=50000):
    total_rows = data.count()
    indexed_data = data.withColumn("row_number", row_number().over(Window.orderBy("some_column")))
    
    for start in range(0, total_rows, chunk_size):
        end = start + chunk_size
        chunk = data.limit(end).offset(start)  
        # chunk = data.limit(end).subtract(data.limit(start))
        chunk.write \
            .mode("overwrite") \
            .option("compression", "snappy") \
            .parquet(db_path)

In [10]:
test_cctv_df = cctv_df.limit(50000)

In [11]:
save_parquet(test_cctv_df, cctv_parquet_path)

In [9]:
save_parquet_in_chunks(cctv_df, cctv_parquet_path)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_com

Py4JError: An error occurred while calling o119.count

In [12]:
test_path = './preprocessed_data/cctv_data'
test_parquet = sdf.read_parquet(test_path)

In [13]:
test_parquet.count()

50000

In [14]:
test_parquet.limit(10).show()

+--------------------+--------------------+--------------------+------------+---------+
|           full_path|             file_id|           file_name| folder_name|file_size|
+--------------------+--------------------+--------------------+------------+---------+
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   741101|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   345849|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1621884|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   740844|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   342661|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1598016|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|

In [11]:
cctv_df.limit(10).show()

+--------------------+--------------------+--------------------+------------+---------+
|           full_path|             file_id|           file_name| folder_name|file_size|
+--------------------+--------------------+--------------------+------------+---------+
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   741101|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   345849|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1621884|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1967217|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   740844|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|   342661|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|  1598016|
|/home/jovyan/work...|0101011112(1138)(...|0101011112(1138)(...|1.고압전동기|

In [16]:
# def save_parquet_in_chunks(data, db_path, chunk_size=10000):
#     total_rows = data.count()
#     for start in range(0, total_rows, chunk_size):
#         try:
#             end = start + chunk_size
#             # chunk = data.limit(end).offset(start)  
#             chunk = data.limit(end).subtract(data.limit(start))
#             chunk.write \
#                 .mode("overwrite") \
#                 .option("compression", "snappy") \
#                 .parquet(db_path)
#         except EOFerror:
#             break
def save_parquet_in_chunks(data, db_path, chunk_size=50000):

    total_rows = data.count()
    num_chunks = (total_rows + chunk_size - 1) // chunk_size

    for i in range(num_chunks):
        start = i * chunk_size
        end = min(start + chunk_size, total_rows)

        # Extract the chunk of data
        chunk = data.limit(end).subtract(data.limit(start))

        # Write the chunk to parquet
        chunk.write \
            .mode("append") \
            .option("compression", "snappy") \
            .parquet(db_path)


In [17]:
save_parquet_in_chunks(cctv_df, cctv_parquet_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 43578)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.

Py4JError: An error occurred while calling o119.count

In [10]:
def save_parquet_in_chunks(data, db_path, chunk_size=100000):
    
    for start in range(0, data.count(), chunk_size):
        chunk = data.limit(chunk_size).offset(start)        
        save_parquet(chunk, db_path)

In [10]:
cctv_df.limit(2).show()

ConnectionRefusedError: [Errno 111] Connection refused

In [12]:
save_parquet_in_chunks(cctv_df, cctv_parquet_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 36196)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

Py4JError: An error occurred while calling o119.count

In [11]:
save_parquet_in_chunks(cctv_df, cctv_parquet_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 42242)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.

Py4JError: An error occurred while calling o119.count

In [11]:
import duckdb

In [ ]:
def save_to_duckdb_in_chunks(data, db_path, table_name, chunk_size=100000):
    # DuckDB 연결
    con = duckdb.connect(db_path)
    
    # 첫 번째 청크 저장 여부를 결정하기 위한 플래그
    first_chunk = True
    
    # Spark DataFrame의 row를 chunk_size 단위로 나눔
    for start in range(0, data.count(), chunk_size):
        # 청크 추출
        chunk = data.limit(chunk_size).offset(start).toPandas()
        
        if first_chunk:
            # 첫 번째 청크는 테이블을 생성하면서 삽입
            con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM chunk")
            first_chunk = False
        else:
            # 이후 청크들은 데이터만 삽입
            con.execute(f"INSERT INTO {table_name} SELECT * FROM chunk")
    
    # DuckDB 연결 종료
    con.close()

In [11]:
%%time
save_parquet(cctv_df, cctv_parquet_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60372)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

Py4JError: An error occurred while calling o237.parquet

In [12]:
%%time
save_parquet(cctv_label_df, cctv_label_parquet_path)

ConnectionRefusedError: [Errno 111] Connection refused

In [13]:
%%time
save_parquet(sand_df, snad_parquet_path)

ConnectionRefusedError: [Errno 111] Connection refused

In [14]:
%%time
save_parquet(sand_label_df, sand_label_parquet_path)

ConnectionRefusedError: [Errno 111] Connection refused

In [8]:
snad_parquet_path = './preprocessed_data/sand_data'
if os.path.exists(snad_parquet_path) == False:
    os.makedirs(snad_parquet_path)

sand_df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(snad_parquet_path)

In [ ]:
sand_label_parquet_path = './preprocessed_data/sand_label_data'
if os.path.exists(sand_label_parquet_path) == False:
    os.makedirs(sand_label_parquet_path)

sand_label_df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(sand_label_parquet_path)

In [ ]:
cctv_parquet_path = './preprocessed_data/cctv_data'
if os.path.exists(snad_parquet_path) == False:
    os.makedirs(cctv_parquet_path)

cctv_df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(cctv_parquet_path)

In [ ]:
cctv_label_parquet_path = './preprocessed_data/cctv_label_data'
if os.path.exists(snad_parquet_path) == False:
    os.makedirs(cctv_label_parquet_path)

cctv_label_df.write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(cctv_label_parquet_path)

In [ ]:
jdbcDF2.write \
    .jdbc("jdbc:postgresql:dbserver", "schema.postgres",
          properties={"postgres": "postgres", "1234": "1234"})

In [ ]:
df_from_parquet.write \
    .jdbc(url=jdbc_url, table="your_table", mode="overwrite", properties=db_properties)

In [12]:
# parquet_data = limited_ten_sand_df.toPandas()

In [13]:
# parquet_data.to_parquet(full_parquet_path, engine='pyarrow')
# parquet_data.to_parquet(full_parquet_path, engine='pyarrow', compression='gzip', index=False)

In [14]:
# test_parquet = pd.read_parquet(full_parquet_path)

### Save database

In [15]:
# import sqlite3
# conn = sqlite3.connect('test.db')
# cursor = conn.cursor()

In [15]:
import psycopg2
from psycopg2 import sql

conn = psycopg2.connect(
    dbname="postgres",  # 초기 연결은 일반적으로 기본 데이터베이스로
    user="postgres",
    password="1234",
    host="postgres",
    port="5432"
)
conn.autocommit = True  # 데이터베이스 생성 시 autocommit 설정

# 커서 생성
cursor = conn.cursor()

In [11]:


# 데이터베이스 생성 쿼리 실행
# database_name = "testdb"
# cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(database_name)))

In [16]:
# 테이블 생성 SQL 명령어
create_table_query = sql.SQL("""
CREATE TABLE IF NOT EXISTS file_data (
    full_path TEXT NOT NULL,
    file_id VARCHAR(20) PRIMARY KEY,
    file_name VARCHAR(255) NOT NULL,
    folder_name VARCHAR(255) NOT NULL,
    file_size BIGINT NOT NULL
)
""")

cursor.execute(create_table_query)

# 변경 사항 커밋
conn.commit()

In [17]:
# url = "jdbc:postgresql://localhost:5432/industry"
url = "jdbc:postgresql://postgres:5432/postgres"

properties = {
    "user": "postgres",
    "password": "1234",
    "driver": "org.postgresql.Driver"
}

sand_df.write.jdbc(url=url, table="file_data", mode="overwrite", properties=properties)

In [18]:
cursor.execute("SELECT * FROM file_data")

In [19]:
cursor.fetchall()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 42728)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.

[('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_1.모래입자크기분류_1/0010002.png',
  '0010002',
  '0010002.png',
  'TS_1.모래입자크기분류_1',
  3156156),
 ('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_1.모래입자크기분류_1/0010003.png',
  '0010003',
  '0010003.png',
  'TS_1.모래입자크기분류_1',
  3029314),
 ('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_1.모래입자크기분류_1/0010004.png',
  '0010004',
  '0010004.png',
  'TS_1.모래입자크기분류_1',
  2930369),
 ('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_1.모래입자크기분류_1/0010005.png',
  '0010005',
  '0010005.png',
  'TS_1.모래입자크기분류_1',
  3293061),
 ('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_1.모래입자크기분류_1/0010006.png',
  '0010006',
  '0010006.png',
  'TS_1.모래입자크기분류_1',
  3143184),
 ('/home/jovyan/work/local_src/industry_data/264.건설 모래 품질 관리

In [ ]:
%%time
url = "jdbc:postgresql://postgres:5432/postgres"

properties = {
    "user": "postgres",
    "password": "1234",
    "driver": "org.postgresql.Driver"
}

# DataFrame을 PostgreSQL에 저장
sand_df.write \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", "file_data") \
    .option("batchsize", "5") \
    .option("numPartitions", "10") \
    .mode("overwrite") \
    .options(**properties) \
    .save()

In [ ]:
cursor.execute("SELECT * FROM file_data")

In [ ]:
cursor.fetchall()

### Test

In [34]:
df = pd.read_parquet(full_parquet_path)

In [49]:
try:
    df.to_sql('parquet_data', connection, if_exists='replace', index=False)
    connection.commit()
except Exception as e:
    print("SQL에 쓰는 중 오류 발생:", e)

In [ ]:
cursor.execute("SELECT * FROM file_data WHERE type='table';")
tables = cursor.fetchall()
print(tables)  # Check if 'parquet_data' is listed


In [51]:
try:
    cursor.execute('SELECT * FROM parquet_data')
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except sqlite3.OperationalError as e:
    print("OperationalError:", e)

('./sand_data/TS_1.모래입자크기분류_1/0010067.png', '0010067', '0010067.png', 'TS_1.모래입자크기분류_1', 3018192)
('./sand_data/TS_1.모래입자크기분류_1/0010049.png', '0010049', '0010049.png', 'TS_1.모래입자크기분류_1', 3287386)
('./sand_data/TS_1.모래입자크기분류_1/0010021.png', '0010021', '0010021.png', 'TS_1.모래입자크기분류_1', 3286182)
('./sand_data/TS_1.모래입자크기분류_1/0010004.png', '0010004', '0010004.png', 'TS_1.모래입자크기분류_1', 2930369)
('./sand_data/TS_1.모래입자크기분류_1/0010094.png', '0010094', '0010094.png', 'TS_1.모래입자크기분류_1', 3266231)
('./sand_data/TS_1.모래입자크기분류_1/0010005.png', '0010005', '0010005.png', 'TS_1.모래입자크기분류_1', 3293061)
('./sand_data/TS_1.모래입자크기분류_1/0010006.png', '0010006', '0010006.png', 'TS_1.모래입자크기분류_1', 3143184)
('./sand_data/TS_1.모래입자크기분류_1/0010057.png', '0010057', '0010057.png', 'TS_1.모래입자크기분류_1', 3275718)
('./sand_data/TS_1.모래입자크기분류_1/0010015.png', '0010015', '0010015.png', 'TS_1.모래입자크기분류_1', 3023326)
('./sand_data/TS_1.모래입자크기분류_1/0010047.png', '0010047', '0010047.png', 'TS_1.모래입자크기분류_1', 3017624)


In [11]:
# read json
json_data = sdf.get_spark_json(sand_label_paths)

In [12]:
prepro.get_folder_name(cctv_paths[0])

'4.계기용변압기(PT)'

In [8]:
pprint(cctv_data_info)
pprint(sand_data_info)
pprint(sand_label_data_info)

('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 1457336 \n'
 '            png counts: 0 \n'
 '            jpg counts: 1093002 \n'
 '            csv counts: 364334\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 490755 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 0 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 49